In [1]:
%pip install setuptools


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import sys
print(sys.executable)  # Check which Python executable is being used
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import layers, models

c:\Users\luisg\Documents\AI\animal-Recognition\server\.venv\Scripts\python.exe
2.18.0


In [5]:
data_dir = "./amazon-rainforest-images/"

batch_size = 32
img_height = 180
img_width = 180

In [6]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 245 files belonging to 11 classes.
Using 196 files for training.


In [7]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 245 files belonging to 11 classes.
Using 49 files for validation.


In [8]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [9]:
num_classes = 12  # Number of animal classes

model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

c:\Users\luisg\Documents\AI\animal-Recognition\server\.venv\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 874ms/step - accuracy: 0.1114 - loss: 2.5964 - val_accuracy: 0.3265 - val_loss: 2.1944
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 812ms/step - accuracy: 0.2084 - loss: 2.2067 - val_accuracy: 0.1633 - val_loss: 2.1599
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 846ms/step - accuracy: 0.2429 - loss: 2.1510 - val_accuracy: 0.3673 - val_loss: 2.0869
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 860ms/step - accuracy: 0.2564 - loss: 2.0814 - val_accuracy: 0.1837 - val_loss: 2.2670
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 855ms/step - accuracy: 0.2332 - loss: 2.0163 - val_accuracy: 0.2857 - val_loss: 2.0541
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 878ms/step - accuracy: 0.2800 - loss: 2.0360 - val_accuracy: 0.3265 - val_loss: 1.9742
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 855ms/step - accuracy: 0.2435 - loss: 1.9655 - val_accuracy: 0.1837 - val_loss: 2.2115
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 924ms/step - accuracy: 0.2962 - loss: 1.9132 - val_accuracy: 0.3061 - val_loss:

In [12]:
val_loss, val_acc = model.evaluate(val_ds)
print(f'Validation accuracy: {val_acc}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - accuracy: 0.2978 - loss: 2.0298
Validation accuracy: 0.30612245202064514


In [14]:
model.save('animal_classifier_model.keras')